<a href="https://colab.research.google.com/github/adesam146/nlpcw/blob/sam_preprocessing/NLP_CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install allennlp

  Using cached https://files.pythonhosted.org/packages/f7/d2/e07d3ebb2bd7af696440ce7e754c59dd546ffe1bbe732c8ab68b9c834e61/cycler-0.10.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/44/72/16630c3392eba03788ad87949390516bbc488e8e118047a3b824631d21a6/kiwisolver-1.0.1-cp36-none-win_amd64.whl
  Running setup.py install for ujson: started
    Running setup.py install for ujson: finished with status 'done'
  Running setup.py install for dill: started
    Running setup.py install for dill: finished with status 'done'
  Running setup.py install for toolz: started
    Running setup.py install for toolz: finished with status 'done'
  Running setup.py install for cytoolz: started
    Running setup.py install for cytoolz: finished with status 'done'
  Found existing installation: colorama 0.4.1
    Uninstalling colorama-0.4.1:
      Successfully uninstalled colorama-0.4.1
  Running setup.py install for numpydoc: started
    Running setup.py install for numpydoc: finis

  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
You are using pip version 9.0.1, however version 19.0.2 is avail

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset

import numpy as np
import pandas as pd

import torchvision.transforms as T

from allennlp.modules.elmo import Elmo, batch_to_ids


In [15]:
#Use pretrained ELMO weights. 
options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

elmo = Elmo(options_file, weight_file, 2, dropout=0)

02/12/2019 20:50:10 - INFO - allennlp.modules.elmo -   Initializing ELMo


In [17]:
#Elmo test
sentences = [['First', 'sentence', '.'], ['Another', '.'], 
             ["Oh", "here", "we", "Go", "now", "you", "fool", "."], 
             ["meaninglesswordnotinvocab"]]
             
character_ids = batch_to_ids(sentences)

# The shape would be (#sentences, length_of_longest_sentence, len of longest word(in original corpus?)=50)
# Note if a word has n characters the first n+2 entries in its size 50 embedding would be used to
# represent it. '+2' because it seems like they implicitly add a start and end of word
# 'character'
print(character_ids.shape)

embeddings = elmo(character_ids)
print(embeddings.keys())
embed = embeddings["elmo_representations"]

# The length of embed is given by the num_output_representations we specfied
# when creating the ELMO class, I believe this are just the number of different
# weighted combination of the 3 layers in the "ELMO network" so could theortically
# be any integer but is usually 1 or 2
print(len(embed))
print(embed[0].shape)
print(embed[1].shape)


torch.Size([4, 8, 50])
dict_keys(['elmo_representations', 'mask'])
2
torch.Size([4, 8, 1024])
torch.Size([4, 8, 1024])


In [0]:
# ELMO takes a list of parsed sentences as an input
# It generates an embedding of length 1024 per word
# We then need to find a good method of combining the word vecs to create 
# a sentence embedding (this article is good: https://medium.com/huggingface/universal-word-sentence-embeddings-ce48ddc8fc3a). 


In [0]:
from google.colab import files

#File upload instructions:
  #1. Run this cell
  #2. Press "Choose Files" at the bottom of this cell
  #3. Select offenseval-training-v1.tsv locally to load it as the variable noisy_values  
  
file1 = files.upload()


Saving offenseval-training-v1.tsv to offenseval-training-v1 (1).tsv


In [0]:
train = pd.read_csv("offenseval-training-v1.tsv", delimiter="\t")
print(train.shape)
print(train.head())

(13240, 5)
      id                                              tweet subtask_a  \
0  86426  @USER She should ask a few native Americans wh...       OFF   
1  90194  @USER @USER Go home you’re drunk!!! @USER #MAG...       OFF   
2  16820  Amazon is investigating Chinese employees who ...       NOT   
3  62688  @USER Someone should'veTaken" this piece of sh...       OFF   
4  43605  @USER @USER Obama wanted liberals &amp; illega...       NOT   

  subtask_b subtask_c  
0       UNT       NaN  
1       TIN       IND  
2       NaN       NaN  
3       UNT       NaN  
4       NaN       NaN  
